In [ ]:
from pycparser import c_parser

import pandas as pd

parser = c_parser.CParser()

root = 'data/'
language = 'c/'
source = pd.read_pickle(root+language+'programs.pkl')
#print(source)
# label是指该段源代码来自于哪个类别吗？
source.columns = ['id', 'code', 'label']
print('parse starting')
source['code'] = source['code'].apply(parser.parse)
source.to_pickle(root+'c/ast.pkl')
print('parse ending')
#source_code = source['code'].iloc[0]
#print(type(source_code))
#ast= parser.parse(source_code)
#ast.show()

In [12]:
import javalang
def parse_program(func):
    tokens = javalang.tokenizer.tokenize(func)
    #print(f'tokens化得到的序列:{tokens}')
    parser = javalang.parser.Parser(tokens)
    tree = parser.parse_member_declaration()
    return tree



In [ ]:
""" func = '''public static int[] bubbleSort(int... a) {
        boolean swapped;
        do {
            swapped = false;
            for (int i = 0; i < a.length - 1; i++) {
                if (a[i] > a[i + 1]) {
                    int tmp = a[i];
                    a[i] = a[i + 1];
                    a[i + 1] = tmp;
                    swapped = true;
                }
            }
        } while (swapped);
        return a;
    }

'''
ast = parse_program(func) """
# for path, node in ast:
#   print(path, node)

In [13]:
from pycparser import c_parser

import pandas as pd

root = 'data/'
language = 'java/'
source_java = pd.read_csv(root+language+'bcb_funcs_all.tsv',
                            sep='\t', 
                            header=None, 
                            encoding='utf-8')
source_java.columns = ['id', 'code']
print('parse starting')
source_java['code'] = source_java['code'].apply(parse_program)
source_java.to_pickle('data/java/ast.pkl')
print('parse ending')
#java = source_java['code'].iloc[2]
#print(java)
#source['code'] = source['code'].apply(parse_program)



parse starting
parse ending


In [1]:
### 分割pairs 胃训练集、验证集、测试集

In [5]:
from pathlib import Path

def split_data(root = 'data/', ratio = '3:1:1', language = 'c'):
  if language is 'c':
    filename = 'oj_clone_ids.pkl'
  else:
    filename = 'bcb_pair_ids.pkl'
  path = root + language + '/' + filename
  data = pd.read_pickle(path)
  data_num = len(data)
  ratios = [int(r) for r in ratio.split(':')]
  train_split = int(ratios[0]/sum(ratios)*data_num)
  val_split = train_split + int(ratios[1]/sum(ratios)*data_num)
  
  #对要分割的数据进行随机打散
  print('starting split')
  data = data.sample(frac=1, random_state=666)

  train = data.iloc[:train_split]
  dev = data.iloc[train_split:val_split]
  test = data.iloc[val_split:]

  def check_or_create(path):
    if not os.path.exists(path):
      os.mkdir(path) 
  train_path = root+language+'/train/'
  check_or_create(train_path)
  assert Path(train_path).is_dir()
  train_file_path = train_path + 'train.pkl'
  train.to_pickle(train_file_path)

  dev_path = root+language+'/dev/'
  check_or_create(dev_path)
  dev_file_path = dev_path + 'dev.pkl'
  train.to_pickle(dev_file_path)

  test_path = root+language+'/test/'
  check_or_create(test_path)
  test_file_path = test_path + 'test.pkl'
  test.to_pickle(test_file_path)
  print('end split')



In [6]:
split_data(language='c')
split_data(language = 'java')

starting split
end split
starting split
end split


In [7]:
language = 'c'
train_file_path = 'clone/data/'+language+'/train/train.pkl'
dev_file_path = 'clone/data/'+language+'/dev/train.pkl'
test_file_path = 'clone/data/'+language+'/test/train.pkl'

In [8]:
import pandas as pd
def read_ast(language):
  root = 'data/'
  source = pd.read_pickle(root+language+'/ast.pkl')
  print(len(source['code']))
  return source
  # ast = source['code'].iloc[1]
  # ast.show()

In [ ]:
#source = read_ast('c')
source_c_ast_DF = read_ast('c')
#source_java_ast_DF = read_ast('java')

In [9]:
root = 'data/'
language = 'c'
train_file_path = root + language + '/train/train.pkl'
pairs = pd.read_pickle(train_file_path)
# 找到训练集所包含的所有的source code id
train_ids = pairs['id1'].append(pairs['id2']).unique()

In [10]:
train_ids

array([9566, 8200, 3897, ..., 9254, 3971, 39727], dtype=object)

In [11]:
# func函数每次处理ast的当前遍历到的结点。即剩余子树的根节点
def func(root,sequence):
  #print(f'c: partial ast root: {root.__class__.__name__}')
  current = SingleNode(root)
  sequence.append(current.get_token())
  #print(f'sequence: {sequence}')
  for _, child in root.children():
      func(child, sequence)
  if current.get_token().lower() == 'compound':
      sequence.append('End')



In [13]:
def trans_to_sequences(ast):
    sequence = []
    func(ast, sequence)
    return sequence

In [13]:
# 设置ast_DFs的索引为id
ast_DFs = source_c_ast_DF.set_index('id',drop=False).loc[train_ids]
#print(ast_DFs)

asts = ast_DFs['code']
sequence = []
corpus = asts.apply(trans_to_sequences)
print(corpus)

id
9566     [FileAST, Decl, ArrayDecl, a, int, 200, Decl, ...
8200     [FileAST, FuncDef, Decl, FuncDecl, main, int, ...
3897     [FileAST, FuncDef, Decl, FuncDecl, main, int, ...
12786    [FileAST, FuncDef, Decl, FuncDecl, main, int, ...
34502    [FileAST, Decl, ArrayDecl, ArrayDecl, map, int...
                               ...                        
29794    [FileAST, Decl, a, int, 0, Decl, i, int, 0, De...
49967    [FileAST, FuncDef, Decl, FuncDecl, main, int, ...
9254     [FileAST, FuncDef, Decl, FuncDecl, main, int, ...
3971     [FileAST, FuncDef, Decl, FuncDecl, main, int, ...
39727    [FileAST, FuncDef, Decl, FuncDecl, main, int, ...
Name: code, Length: 7498, dtype: object


In [21]:
#print(type(corpus)) # Series
str_corpus = [' '.join(c) for c in corpus]
ast_seq = source_c_ast_DF.copy(False)
ast_seq['code'] = pd.Series(str_corpus)
ast_seq.to_pickle(root+language+'/train/ast_seq.pkl')

In [9]:
# 使用word2vec从训练集中训练一个词典
from gensim.models.word2vec import Word2Vec
embedding_size = 128
w2v = Word2Vec(corpus, size=embedding_size, workers=16, sg=1, max_final_vocab=3000)
w2v.save(root+language+'/train/embedding_astnode_w2v_' + str(embedding_size))

NameError: name 'corpus' is not defined

In [8]:
import sys
sys.path.append('..')
from tree import ASTNode, SingleNode 
from prepare_data import get_blocks

In [7]:
# generate block sequences with index representations
from gensim.models.word2vec import Word2Vec
embedding_size = 128
def generate_block_seqs():
    word2vec = Word2Vec.load(root+language+'/train/embedding_astnode_w2v_' + str(embedding_size)).wv
    vocab = word2vec.vocab
    max_token = word2vec.syn0.shape[0]
    
    # 这个递归函数用来干什么？
    def tree_to_index(node):
        token = node.token
        result = [vocab[token].index if token in vocab else max_token]
        children = node.children
        for child in children:
            #print(f'tree_to_index: a node''s child: {child.token}')
            result.append(tree_to_index(child))
        return result

    def trans2seq(ast):
        blocks = []
        get_blocks(ast, blocks)
        tree = []
        for b in blocks:
            btree = tree_to_index(b)
            tree.append(btree)
        return tree
    # source_c_ast_DF就是id code(ast形式) label 
    source_2_seq_DF =  source_c_ast_DF.copy(False)
    trees = source_2_seq_DF['code']
    # 确认trees copy成功
    #print(trees)
    # trans2seq操作是在ast上进行的
    # 把每一个ast转换成seq blocks
    # print(trees['code'].iloc[1])
    source_2_seq_DF['code'] = trees.apply(trans2seq)
    if 'label' in source_2_seq_DF.columns:
        source_2_seq_DF.drop('label', axis=1, inplace=True)
    blocks = source_2_seq_DF


In [ ]:
generate_block_seqs()

In [6]:
#generate_block_seqs()
blocks = []
get_blocks(ast_for_block,blocks)
#print(blocks)


NameError: name 'get_blocks' is not defined

In [12]:
from gensim.models.word2vec import Word2Vec
embedding_size = 128
root='data/'
language = 'c'
word2vec = Word2Vec.load(root+language+'/train/embedding_astnode_w2v_' + str(embedding_size)).wv
vocab = word2vec.vocab
max_token = word2vec.syn0.shape[0]

def tree_to_index(node):
    token = node.token
    result = [vocab[token].index if token in vocab else max_token]
    children = node.children
    for child in children:
        print(f'tree_to_index: a node  child: {child.token}')
        result.append(tree_to_index(child))
    return result
tree = []
for b in blocks:
    print(b.token)
    btree = tree_to_index(b)
    print(btree)
    tree.append(btree)
print(tree)

FuncDef
tree_to_index: a node  child: Decl
tree_to_index: a node  child: FuncDecl
tree_to_index: a node  child: main
tree_to_index: a node  child: int
[30, [1, [29, [40, [2]]]]]
Compound
[6]
Decl
tree_to_index: a node  child: ArrayDecl
tree_to_index: a node  child: s
tree_to_index: a node  child: char
tree_to_index: a node  child: 256
[1, [21, [34, [50]], [138]]]
Decl
tree_to_index: a node  child: ArrayDecl
tree_to_index: a node  child: a
tree_to_index: a node  child: char
tree_to_index: a node  child: 256
[1, [21, [12, [50]], [138]]]
Decl
tree_to_index: a node  child: ArrayDecl
tree_to_index: a node  child: b
tree_to_index: a node  child: char
tree_to_index: a node  child: 256
[1, [21, [24, [50]], [138]]]
Decl
tree_to_index: a node  child: l1
tree_to_index: a node  child: int
[1, [163, [2]]]
Decl
tree_to_index: a node  child: i
tree_to_index: a node  child: int
[1, [0, [2]]]
Decl
tree_to_index: a node  child: j
tree_to_index: a node  child: int
[1, [8, [2]]]
Decl
tree_to_index: a node

In [3]:
lens = [len(item) for item in tree]
        max_len = max(lens)

IndentationError: unexpected indent (<ipython-input-3-d6d65541cb23>, line 2)

In [13]:
for b in blocks:
    print(b.token)
    children = b.children
    child_list = [c.token for c in children]
    print(child_list)

FuncDef
['Decl']
Compound
[]
Decl
['ArrayDecl']
Decl
['ArrayDecl']
Decl
['ArrayDecl']
Decl
['l1']
Decl
['i']
Decl
['j']
Decl
['n']
Decl
['flag']
FuncCall
['scanf', 'ExprList']
FuncCall
['scanf', 'ExprList']
FuncCall
['scanf', 'ExprList']
=
['l1', 'FuncCall']
For
['=', '!=', '++']
Compound
[]
If
['==']
Compound
[]
=
['flag', '1']
For
['=', '<', '++']
Compound
[]
If
['!=']
=
['flag', '0']
=
['flag', '1']
End
[]
If
['==']
Compound
[]
For
['=', '<', '++']
FuncCall
['printf', 'ExprList']
FuncCall
['printf', 'ExprList']
For
['=', '!=', '++']
FuncCall
['printf', 'ExprList']
Return
['0']
End
[]
End
[]
End
[]
FuncCall
['printf', 'ExprList']
Return
['0']
End
[]


In [4]:
code = '''int main()
{
	char s[256],a[256],b[256];
	int l1,i,j,n;
	int flag;
	scanf("%s",s);
	scanf("%s",a);
	scanf("%s",b);
	l1=strlen(a);
	for(i=0;s[i]!='\0';i++)
	{
		if(s[i]==a[0])
		{
			flag=1;
			for(j=i+1;j<(i+l1);j++)
			{
				if (s[j]!=a[j-i])
					flag=0;
				else 
				    flag=1;
			}
			if(flag==1)
			{
				for(n=0;n<i;n++)
					printf("%c",s[n]);
				printf("%s",b);
				for(n=i+l1;s[n]!='\0';n++)
					printf("%c",s[n]);
				return 0;
			}
		}
	}
	printf("%s",s);
	return 0;
}
'''
#print(code)

In [5]:
from pycparser import c_parser

import pandas as pd

parser = c_parser.CParser()

ast_for_block = parser.parse(code)
print(ast_for_block)

In [28]:
source_c_ast_DF

,id,code,label
0,0,<pycparser.c_ast.FileAST object at 0x000001AAA...,97
1,1,<pycparser.c_ast.FileAST object at 0x000001AAA...,97
2,2,<pycparser.c_ast.FileAST object at 0x000001AAA...,97
3,3,<pycparser.c_ast.FileAST object at 0x000001AAA...,97
4,4,<pycparser.c_ast.FileAST object at 0x000001AAA...,97
...,...,...,...
51996,51996,<pycparser.c_ast.FileAST object at 0x000001AC2...,72
51997,51997,<pycparser.c_ast.FileAST object at 0x000001AC2...,72
51998,51998,<pycparser.c_ast.FileAST object at 0x000001AC2...,72
51999,51999,<pycparser.c_ast.FileAST object at 0x000001AC2...,72
